# Transfer Learning Model 'Xception' #

In [1]:

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.applications.xception import Xception


Using TensorFlow backend.


In [4]:
def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     train_path = os.path.join(base_dir, 'train')
     valid_path = os.path.join(base_dir, 'valid')

     # instantiate data generators
     datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

     train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary')

     return train_gen, val_gen

# the size of the images in the PCAM dataset
IMAGE_SIZE = 96
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
train_gen, val_gen = get_pcam_generators('C:/Users/s165635/Desktop/Project Imaging (8P361)/Dataset CHAMELYON16')
input = Input(input_shape)
# get the pretrained model, cut out the top layer
pretrained = Xception(input_shape=input_shape, include_top=False, weights='imagenet')

#for layer in pretrained.layers[-4]:
#    layer.trainable = False

# Freeze the layers except the last 4 layers
#for i, layer in enumerate(pretrained.layers):
    #layer.trainable = False
#    print(layer.name)

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [ ]:
for layer in pretrained.layers:
            layer.trainable = False
        
layernames = []
for i,layer in enumerate(pretrained.layers):
    layernames.append(layer.name)
    

for j in range(14,0,-1):
    string = 'block'+str(j)
   # print(string)

    indices = [i for i, s in enumerate(layernames) if string in s]
        
    for layer in pretrained.layers[min(indices):]:
        layer.trainable = True
    


    output = pretrained(input)
    output = GlobalAveragePooling2D()(output)
    output = Dropout(0.5)(output)
    output = Dense(1, activation='sigmoid')(output)

    # Check the trainable status of the individual layers
    #for layer in pretrained.layers:
    #    print(layer, layer.trainable)

    model = Model(input, output)

    # note the lower lr compared to the cnn example
    model.compile(SGD(lr=0.001, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])
    #
    # print a summary of the model on screen
    model.summary()
        
    # save the model and weights
    model_name = 'Xception weights ImageNet'+string
    model_filepath = model_name + '.json'
    weights_filepath = model_name + '_weights.hdf5'

    model_json = model.to_json() # serialize model to JSON
    with open(model_filepath, 'w') as json_file:
        json_file.write(model_json)


    # define the model checkpoint and Tensorboard callbacks
    checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    tensorboard = TensorBoard(os.path.join('logs', model_name))
    callbacks_list = [checkpoint, tensorboard]


    # train the model, note that we define "mini-epochs"
    train_steps = train_gen.n//train_gen.batch_size//20
    val_steps = val_gen.n//val_gen.batch_size//20

    # since the model is trained for only 10 "mini-epochs", i.e. half of the data is
    # not used during training

    history = model.fit_generator(train_gen, steps_per_epoch=train_steps,
                        validation_data=val_gen,
                        validation_steps=val_steps,
                        epochs=10,
                        callbacks=callbacks_list)

    # print a summary of the model on screen
    model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2049      
Total params: 20,863,529
Trainable params: 4,750,849
Non-trainable params: 16,112,680
_________________________________________________________________
Epoch 1/10
 33/225 [===>..........................] - ETA: 7:46 - loss: 0.7092 - acc: 0.4867

In [45]:
for layer in pretrained.layers:
            layer.trainable = False
        
layernames = []
for i,layer in enumerate(pretrained.layers):
    layernames.append(layer.name)
    

for j in range(14,0,-1):
    string = 'block'+str(j)
   # print(string)

    indices = [i for i, s in enumerate(layernames) if string in s]
        
    for layer in pretrained.layers[min(indices):]:
        layer.trainable = True
    
    for i,layer in enumerate(pretrained.layers):
        print(string,layer.name,layer.trainable)
        

block14 input_2 False
block14 block1_conv1 False
block14 block1_conv1_bn False
block14 block1_conv1_act False
block14 block1_conv2 False
block14 block1_conv2_bn False
block14 block1_conv2_act False
block14 block2_sepconv1 False
block14 block2_sepconv1_bn False
block14 block2_sepconv2_act False
block14 block2_sepconv2 False
block14 block2_sepconv2_bn False
block14 conv2d_1 False
block14 block2_pool False
block14 batch_normalization_1 False
block14 add_1 False
block14 block3_sepconv1_act False
block14 block3_sepconv1 False
block14 block3_sepconv1_bn False
block14 block3_sepconv2_act False
block14 block3_sepconv2 False
block14 block3_sepconv2_bn False
block14 conv2d_2 False
block14 block3_pool False
block14 batch_normalization_2 False
block14 add_2 False
block14 block4_sepconv1_act False
block14 block4_sepconv1 False
block14 block4_sepconv1_bn False
block14 block4_sepconv2_act False
block14 block4_sepconv2 False
block14 block4_sepconv2_bn False
block14 conv2d_3 False
block14 block4_pool F

block9 block9_sepconv1_bn True
block9 block9_sepconv2_act True
block9 block9_sepconv2 True
block9 block9_sepconv2_bn True
block9 block9_sepconv3_act True
block9 block9_sepconv3 True
block9 block9_sepconv3_bn True
block9 add_8 True
block9 block10_sepconv1_act True
block9 block10_sepconv1 True
block9 block10_sepconv1_bn True
block9 block10_sepconv2_act True
block9 block10_sepconv2 True
block9 block10_sepconv2_bn True
block9 block10_sepconv3_act True
block9 block10_sepconv3 True
block9 block10_sepconv3_bn True
block9 add_9 True
block9 block11_sepconv1_act True
block9 block11_sepconv1 True
block9 block11_sepconv1_bn True
block9 block11_sepconv2_act True
block9 block11_sepconv2 True
block9 block11_sepconv2_bn True
block9 block11_sepconv3_act True
block9 block11_sepconv3 True
block9 block11_sepconv3_bn True
block9 add_10 True
block9 block12_sepconv1_act True
block9 block12_sepconv1 True
block9 block12_sepconv1_bn True
block9 block12_sepconv2_act True
block9 block12_sepconv2 True
block9 block

block4 add_8 True
block4 block10_sepconv1_act True
block4 block10_sepconv1 True
block4 block10_sepconv1_bn True
block4 block10_sepconv2_act True
block4 block10_sepconv2 True
block4 block10_sepconv2_bn True
block4 block10_sepconv3_act True
block4 block10_sepconv3 True
block4 block10_sepconv3_bn True
block4 add_9 True
block4 block11_sepconv1_act True
block4 block11_sepconv1 True
block4 block11_sepconv1_bn True
block4 block11_sepconv2_act True
block4 block11_sepconv2 True
block4 block11_sepconv2_bn True
block4 block11_sepconv3_act True
block4 block11_sepconv3 True
block4 block11_sepconv3_bn True
block4 add_10 True
block4 block12_sepconv1_act True
block4 block12_sepconv1 True
block4 block12_sepconv1_bn True
block4 block12_sepconv2_act True
block4 block12_sepconv2 True
block4 block12_sepconv2_bn True
block4 block12_sepconv3_act True
block4 block12_sepconv3 True
block4 block12_sepconv3_bn True
block4 add_11 True
block4 block13_sepconv1_act True
block4 block13_sepconv1 True
block4 block13_sep

In [35]:
print(string)

block1


In [33]:
for i,layer in enumerate(pretrained.layers):
    print(layer.name, layer.trainable)

input_2 False
block1_conv1 False
block1_conv1_bn False
block1_conv1_act False
block1_conv2 False
block1_conv2_bn False
block1_conv2_act False
block2_sepconv1 False
block2_sepconv1_bn False
block2_sepconv2_act False
block2_sepconv2 False
block2_sepconv2_bn False
conv2d_1 False
block2_pool False
batch_normalization_1 False
add_1 False
block3_sepconv1_act False
block3_sepconv1 False
block3_sepconv1_bn False
block3_sepconv2_act False
block3_sepconv2 False
block3_sepconv2_bn False
conv2d_2 False
block3_pool False
batch_normalization_2 False
add_2 False
block4_sepconv1_act False
block4_sepconv1 False
block4_sepconv1_bn False
block4_sepconv2_act False
block4_sepconv2 False
block4_sepconv2_bn False
conv2d_3 False
block4_pool False
batch_normalization_3 False
add_3 False
block5_sepconv1_act False
block5_sepconv1 False
block5_sepconv1_bn False
block5_sepconv2_act False
block5_sepconv2 False
block5_sepconv2_bn False
block5_sepconv3_act False
block5_sepconv3 False
block5_sepconv3_bn False
add_4 Fa

In [7]:
#for i,layer in enumerate(model.layers):
    #print(i,layer.name)

#for layer in model.layers:
#    print(layer.output_shape)
# get the data generators


# save the model and weights
model_name = 'Xception weights ImageNet 14 off'
model_filepath = model_name + '.json'
weights_filepath = model_name + '_weights.hdf5'

model_json = model.to_json() # serialize model to JSON
with open(model_filepath, 'w') as json_file:
    json_file.write(model_json)


# define the model checkpoint and Tensorboard callbacks
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join('logs', model_name))
callbacks_list = [checkpoint, tensorboard]


# train the model, note that we define "mini-epochs"
train_steps = train_gen.n//train_gen.batch_size//20
val_steps = val_gen.n//val_gen.batch_size//20

# since the model is trained for only 10 "mini-epochs", i.e. half of the data is
# not used during training

history = model.fit_generator(train_gen, steps_per_epoch=train_steps,
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10,
                    callbacks=callbacks_list)

# print a summary of the model on screen
model.summary()

Epoch 1/10
  5/225 [..............................] - ETA: 35:25 - loss: 0.7426 - acc: 0.4875

KeyboardInterrupt: 